# 11.04 - Lazy Evaluation y Optimización

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Avanzado  
**Tiempo estimado:** 50 min

La verdadera potencia de Polars reside en su API *Lazy*. En este lab aprenderemos cómo Polars optimiza nuestras consultas antes de siquiera tocar los datos.

In [1]:
import polars as pl
import numpy as np
import os

# Generar un dataset más grande (5M filas)
n = 5_000_000
df = pl.DataFrame({
    "id": np.arange(n),
    "group": np.random.choice(["A", "B", "C", "D"], n),
    "value": np.random.randn(n)
})

# Guardar como Parquet para demostrar escaneo lazy
df.write_parquet("data_temp.parquet")

## 1. Scan vs Read
- `read_parquet()`: Lee todo el archivo en memoria inmediatamente.
- `scan_parquet()`: Crea un plan de consulta sin leer nada aún.

In [2]:
lazy_df = pl.scan_parquet("data_temp.parquet")
print(type(lazy_df))

# Operaciones en modo Lazy
query = (
    lazy_df
    .filter(pl.col("group") == "A")
    .group_by("group")
    .agg(pl.col("value").mean())
)

print("Query Plan:")
print(query.explain()) # Aquí vemos la optimización de predicados (Predicate Pushdown)

<class 'polars.lazyframe.frame.LazyFrame'>
Query Plan:
AGGREGATE
  [col("value").mean()] BY [col("group")]
  FROM
  Parquet SCAN [data_temp.parquet]
  PROJECT 2/3 COLUMNS
  SELECTION: [(col("group")) == ("A")]


## 2. Optimizaciones del Planificador
Polars realiza varias optimizaciones automáticamente:
- **Projection Pushdown:** Solo lee las columnas necesarias.
- **Predicate Pushdown:** Filtra los datos lo antes posible (incluso a nivel de metadatos del archivo Parquet).
- **Common Subexpression Elimination:** No calcula lo mismo dos veces.

In [3]:
# Ejecutando la consulta con collect()
result = query.collect()
result

group,value
str,f64
"""A""",0.000471


## 3. Escalabilidad
Cuando el dataset no cabe en la RAM, Polars utiliza su motor de ejecución altamente optimizado para procesar los datos de forma eficiente.

In [4]:
# Para archivos masivos fuera de memoria (Out-of-core):
# result = query.collect(streaming=True) 
print("Para procesar archivos gigantes use q.collect(streaming=True)")
result

Para procesar archivos gigantes use q.collect(streaming=True)


group,value
str,f64
"""A""",0.000471


In [5]:
# Limpieza
if os.path.exists("data_temp.parquet"):
    os.remove("data_temp.parquet")

## Resumen
- Usa `scan_*` siempre que sea posible.
- El planificador de Polars es tu mejor aliado para el rendimiento.
- Polars optimiza automáticamente el acceso al disco (IO) y la memoria.

---

**Anterior:** [11.03 - Polars Wrangling](./11_03_polars_wrangling.ipynb)  
**Siguiente:** [11.05 - Polars Time Series](./11_05_polars_timeseries.ipynb)